In [ ]:
# ===================================================================================================
# ============================================ IMPORTS ==============================================
# ===================================================================================================
import numpy as np
from obspy import Trace
import matplotlib.pyplot as plt
import matplotlib.image as img
import matplotlib.gridspec as gridspec
import matplotlib
import obspy
import glob as g
from obspy.clients.fdsn import Client
from obspy import read
import datetime
import math
from obspy.core import UTCDateTime
import cmath
from pathlib import Path
import pandas as pd
from obspy import taup

#just an easy save tool i like
def save_tight(filename,fig=None,dpi=200,format=None):
        # Saves figure to PDF with no margins. Do not modify
        # plt.gca().set_axis_off()
        # plt.subplots_adjust(top = 1, bottom = 0.0, right = 1, left = 0,hspace = 0.07, wspace = 0.03)
        plt.margins(0.1,0.1)
        # plt.gca().xaxis.set_major_locator(plt.NullLocator())
        # plt.gca().yaxis.set_major_locator(plt.NullLocator())
        if fig is None:
                plt.savefig(filename, bbox_inches = 'tight',pad_inches = 0.05,dpi=dpi,format=format)
        else:
                fig.savefig(filename, bbox_inches = 'tight',pad_inches = 0.05,dpi=dpi,format=format)
        return 'Complete'

#useful tool for ambient noise work on OBS
def fnotch(d):
        '''The frequency knotch root function described in Crawford et al., 1998.
        depth (d) is in meters. Returned (f) is in Hz.'''
        g = 9.80665
        f = (g/(2*np.pi*d))**0.5
        return f

#just a nice script to have for quick arrival times
def get_arrivals(sta_llaz,ev_llaz,model = 'iasp91',phases=('ttall',)):
        ''''
        Simple function pulls arrival times and phase names for a given event observed at a given station
        sta_llaz = List object containing [Lat,Lon] of the station
        ev_llaz = List object containing [Lat,Lon] of the event
        phases = Tuple object containing a list of all desired phases. 'ttall'
        (default) will give every single phase available.
        -Charles Hoots,2022
        '''
        degdist = obspy.taup.taup_geo.calc_dist(ev_llaz[0],ev_llaz[1],sta_llaz[0],sta_llaz[1],6371,0)
        arrivals = obspy.taup.tau.TauPyModel(model=model).get_travel_times(ev_llaz[2], degdist,phase_list=phases)
        times = [[a.name,a.time] for a in arrivals]
        ardict = dict()
        _ = [ardict.update({a[0]:a[1]})  for a in times]
        return ardict


In [ ]:

datafolder = Path('/Volumes/scienceparty_share/SR2411_scienceparty_share/SeismicData')
eqlist = Path( '/Volumes/scienceparty_share/SR2411_scienceparty_share/Seimic_data_samples_and_codes/eq_lists/usgs_eq_query_global_5.5plus.csv')
stalist = Path('/Volumes/scienceparty_share/SR2411_scienceparty_share/OBS_List.csv')
events = pd.read_csv(eqlist)
events = events.assign(datetime = [UTCDateTime.strptime(d.replace('T',' ').replace('Z',''),'%Y-%m-%d %H:%M:%S.%f') for d in events.time.iloc])
events = events.assign(eqname = ['eq{d}'.format(d=m.strftime('%Y%m%d%H%M%S')) for m in events.datetime])
events = events.assign(year = [d.strftime('%Y') for d in events.datetime])
events = events.assign(day = [d.strftime('%j') for d in events.datetime])

stations = pd.read_csv(stalist,skiprows=[0])
stations = stations.rename(columns={'Station Name':'Station','OBS Type':'OBS_Type','OBS I.D.':'OBS_ID','Station Depth (m)':'Station_Depth_m'})
stations.Station = [str(s).zfill(2) for s in stations.Station]
stations = stations.assign(Serial = stations.OBS_ID + '_' + stations.Station)

# I just wanted to see >m6.5
events = events[events.mag>6.5]
# events = events[events.mag>7.0]

In [ ]:
station = 'AN106_56'
station_set = stations[stations.Serial==station].copy()

format = 'png' #plot save fmt
resample_fq = 1/5 #decide on a resample frequency
resample_nyq = resample_fq/2 #nyquist, used for anti-alias filter PRIOR to resampling

#bandpass fq
freqmin=1/50
freqmax=1/10


px = 1/plt.rcParams['figure.dpi']  # pixel in inches
spec_figsize = (800*px,1000*px)
# spec_figsize = (12,10)

phases = ('P','S','SKS','SKKS','SKiKS')    
# phases = ('ttall',)
for evi,ev in enumerate(events.iloc):
  if ev.datetime.hour>=22:
    continue
  loadfold = datafolder / station / 'Data' / str(ev.year) / str(ev.day)
  if not loadfold.is_dir():
    continue
  print('**'*20)
  print('{evi} / {n}'.format(evi=evi+1,n=len(events)))
  print('input:{f}'.format(f=loadfold))
  start = ev.datetime
  end = ev.datetime + 3600*2
  st = read(loadfold / '*.msd') #load the data
  
  st.detrend('simple') #demean and detrend the data
  st.detrend('demean')
  

  st.filter('lowpass',freq=resample_nyq,zerophase=True,corners=5)
  st.resample(resample_fq)
  st.filter('bandpass',freqmin=freqmin,freqmax=freqmax)
  st.trim(starttime=start,endtime=end)   #AFTER filter/resamp, trim to event times

  st_trim = st.copy() #making copies are often a smart idea
  

  # Event phase arrival times
  st_laloz = [station_set.Latitude.iloc[0],station_set.Longitude.iloc[0],station_set.Station_Depth_m.iloc[0]/1000]
  ev_laloz = [ev.latitude,ev.longitude,ev.depth]    
  arrivals = get_arrivals(sta_llaz=st_laloz,ev_llaz=ev_laloz,phases=phases)
  
  # Plotting
  h = st_trim.plot(handle=True,title='hello!',equal_scale=False,type='relative')
  axes = h.get_children()[1:-1]
  _ = [[ax.axvline(arrivals[k]) for k in list(arrivals.keys())] for ax in axes]
  _ = [[ax.text(arrivals[k],max(ax.get_ylim())*(0.9**(i)),k) for i,k in  enumerate(list(arrivals.keys())) if k[0]=='P'] for ax in axes]
  _ = [[ax.text(arrivals[k],max(ax.get_ylim())*(0.9**(i)),k) for i,k in  enumerate(list(arrivals.keys())) if k[0]=='S'] for ax in axes]
  deftxt = (h.get_children()[-1])._text
  fname = '{s}_m{m}_z{z}km'.format(s=station,m=ev.mag,z=int(ev.depth))
  h.suptitle(deftxt + '\n' + str(int(1/freqmin)) + '-' + str(int(1/freqmax)) + 's | '+ '(sta-z ' + str(int(st_laloz[-1])) + 'km) ' + fname.replace('_',' | '))
  fold = Path('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/Notebooks/test')
  fold = fold / (fname + '_' + ev.eqname)
  fold.mkdir(parents=True,exist_ok=True)
  fname = fname + '_' + ev.eqname
  save_tight(filename=fold / (fname + '_traces.' + format),fig=h,format=format)
  print('Output: ' + str(fname))
  plt.close('all')
  fig, axes = plt.subplots(nrows=4, ncols=1,figsize=spec_figsize,layout='constrained',squeeze=True,sharey='all',sharex='all')
  comps = ['*Z','*1','*2','*H']
  for axi,s in enumerate(comps):
    cst = st_trim.select(channel=s)[0]
    ax = axes[axi]
    outfile = filename=fold / (fname + '_spec' '.' + format)
    cst.spectrogram(log=False,outfile=outfile,mult=128.0,axes=ax)
    ax.set_xlim([0,3600*2])
    ax.update({'xlim':[0,3600*2]})
    ax.set_title(cst.stats.channel)
    # save_tight(filename=fold / (fname + '_spec_' + s.stats.channel + '.' + format),format=format) #depreciated
  fig.suptitle('(sta-z ' + str(int(st_laloz[-1])) + 'km) ' + fname.replace('_',' | '))
  save_tight(filename=outfile,fig=fig,format=format)
  plt.close('all')
print('Done!!')

In [ ]:
# Should I get the instrumet response? Naahhhhhh....
# A rough code snippet from an old fetch code if I want to:

# import obspy
# from obspy.core.inventory import Inventory, Network, Station, Channel, Site
# from obspy.clients.nrl import NRL
# nrl = NRL()
# response = nrl.get_response( # doctest: +SKIP
#     sensor_keys=['Streckeisen', 'STS-1', '360 seconds'],
#     datalogger_keys=['REF TEK', 'RT 130 & 130-SMA', '1', '200'])


# response